In [2]:
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD

# Defining the model structure. We can define the network by just passing a list of edges.
model = BayesianModel([('D', 'G'), ('I', 'G'), ('G', 'L'),
                       ('I', 'S')])

# Defining individual CPDs.
cpd_d = TabularCPD(variable='D', variable_card=2, 
                   values=[[0.6, 0.4]])
cpd_i = TabularCPD(variable='I', variable_card=2, 
                   values=[[0.7, 0.3]])

cpd_g = TabularCPD(variable='G', variable_card=3, 
                   values=[[0.3, 0.05, 0.9,  0.5],
                           [0.4, 0.25, 0.08, 0.3],
                           [0.3, 0.7,  0.02, 0.2]],
                  evidence=['I', 'D'],
                  evidence_card=[2, 2])

cpd_l = TabularCPD(variable='L', variable_card=2, 
                   values=[[0.1, 0.4, 0.99],
                           [0.9, 0.6, 0.01]],
                   evidence=['G'],
                   evidence_card=[3])

cpd_s = TabularCPD(variable='S', variable_card=2,
                   values=[[0.95, 0.2],
                           [0.05, 0.8]],
                   evidence=['I'],
                   evidence_card=[2])

# Associating the CPDs with the network
model.add_cpds(cpd_d, cpd_i, cpd_g, cpd_l, cpd_s)

print(model.check_model())
print(model.nodes())

True
['D', 'G', 'I', 'L', 'S']


In [4]:
# We can now call some methods on the BayesianModel object.
model.get_cpds()


[<TabularCPD representing P(D:2) at 0x197c82fe390>,
 <TabularCPD representing P(I:2) at 0x197c82fe3c8>,
 <TabularCPD representing P(G:3 | I:2, D:2) at 0x197c82fe4e0>,
 <TabularCPD representing P(L:2 | G:3) at 0x197c82fe400>,
 <TabularCPD representing P(S:2 | I:2) at 0x197ca42d940>]

In [5]:
print(model.get_cpds('G'))

╒═════╤═════╤══════╤══════╤═════╕
│ I   │ I_0 │ I_0  │ I_1  │ I_1 │
├─────┼─────┼──────┼──────┼─────┤
│ D   │ D_0 │ D_1  │ D_0  │ D_1 │
├─────┼─────┼──────┼──────┼─────┤
│ G_0 │ 0.3 │ 0.05 │ 0.9  │ 0.5 │
├─────┼─────┼──────┼──────┼─────┤
│ G_1 │ 0.4 │ 0.25 │ 0.08 │ 0.3 │
├─────┼─────┼──────┼──────┼─────┤
│ G_2 │ 0.3 │ 0.7  │ 0.02 │ 0.2 │
╘═════╧═════╧══════╧══════╧═════╛


In [6]:
model.get_cardinality('G')

3

In [7]:
# Getting the local independencies of a variable.
model.local_independencies('G')

(G _|_ S, L | I, D)

In [8]:
# Getting all the local independencies in the network.
model.local_independencies(['D', 'I', 'S', 'G', 'L'])

(D _|_ S, G, I, L)
(I _|_ S, G, L, D)
(S _|_ G, L, D | I)
(G _|_ S, L | I, D)
(L _|_ S, I, D | G)

In [9]:
from pgmpy.inference import VariableElimination
infer = VariableElimination(model)
print(infer.query(['G']) ['G'])

╒═════╤══════════╕
│ G   │   phi(G) │
╞═════╪══════════╡
│ G_0 │   0.3620 │
├─────┼──────────┤
│ G_1 │   0.2884 │
├─────┼──────────┤
│ G_2 │   0.3496 │
╘═════╧══════════╛


C:\Anaconda3\lib\site-packages\pgmpy\factors\discrete\DiscreteFactor.py:598: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  phi1.values = phi1.values[slice_]
C:\Anaconda3\lib\site-packages\pgmpy\factors\discrete\DiscreteFactor.py:586: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  phi.values = phi.values[slice_]


In [10]:
print(infer.query(['G'], evidence={'D': 0, 'I': 1}) ['G'])

╒═════╤══════════╕
│ G   │   phi(G) │
╞═════╪══════════╡
│ G_0 │   0.9000 │
├─────┼──────────┤
│ G_1 │   0.0800 │
├─────┼──────────┤
│ G_2 │   0.0200 │
╘═════╧══════════╛


In [11]:
print(infer.query(['G'], evidence={'D': 1, 'I': 1,'S':1}) ['G'])

╒═════╤══════════╕
│ G   │   phi(G) │
╞═════╪══════════╡
│ G_0 │   0.5000 │
├─────┼──────────┤
│ G_1 │   0.3000 │
├─────┼──────────┤
│ G_2 │   0.2000 │
╘═════╧══════════╛


In [12]:
print(infer.query(['L'], evidence={'D': 1, 'I': 1,'G':1}) ['L'])

╒═════╤══════════╕
│ L   │   phi(L) │
╞═════╪══════════╡
│ L_0 │   0.4000 │
├─────┼──────────┤
│ L_1 │   0.6000 │
╘═════╧══════════╛
